# Neon Postgresを使用したベクトル類似度検索

このノートブックでは、[Neon Serverless Postgres](https://neon.tech/)をOpenAI埋め込みのベクトルデータベースとして使用する方法を説明します。以下の内容を実演します：

1. OpenAI APIで作成された埋め込みを使用する
2. Neon Serverless Postgresデータベースに埋め込みを保存する
3. OpenAI APIを使用して生のテキストクエリを埋め込みに変換する
4. `pgvector`拡張機能を使用してNeonでベクトル類似度検索を実行する

## 前提条件

開始する前に、以下のものが揃っていることを確認してください：

1. Neon Postgresデータベース。アカウントを作成し、すぐに使える`neondb`データベースを含むプロジェクトを数ステップで設定できます。手順については、[サインアップ](https://neon.tech/docs/get-started-with-neon/signing-up)と[最初のプロジェクトの作成](https://neon.tech/docs/get-started-with-neon/setting-up-a-project)を参照してください。
2. Neonデータベースの接続文字列。Neon **Dashboard**の**Connection Details**ウィジェットからコピーできます。[任意のアプリケーションからの接続](https://neon.tech/docs/connect/connect-from-any-app)を参照してください。
3. `pgvector`拡張機能。`CREATE EXTENSION vector;`を実行してNeonに拡張機能をインストールしてください。手順については、[pgvector拡張機能の有効化](https://neon.tech/docs/extensions/pgvector#enable-the-pgvector-extension)を参照してください。
4. あなたの[OpenAI APIキー](https://platform.openai.com/account/api-keys)。
5. Pythonと`pip`。

### 必要なモジュールのインストール

このノートブックには`openai`、`psycopg2`、`pandas`、`wget`、および`python-dotenv`パッケージが必要です。これらは`pip`でインストールできます：

In [ ]:
! pip install openai psycopg2 pandas wget python-dotenv

### OpenAI APIキーの準備

ドキュメントとクエリのベクトルを生成するには、OpenAI APIキーが必要です。

OpenAI APIキーをお持ちでない場合は、https://platform.openai.com/account/api-keys から取得してください。

OpenAI APIキーをオペレーティングシステムの環境変数として追加するか、プロンプトが表示されたときにセッション用に提供してください。環境変数を定義する場合は、変数名を`OPENAI_API_KEY`としてください。

OpenAI APIキーを環境変数として設定する方法については、[APIキーの安全性に関するベストプラクティス](https://help.openai.com/en/articles/5112595-best-practices-for-api-key-safety)を参照してください。

### OpenAPIキーをテストする

In [1]:
# Test to ensure that your OpenAI API key is defined as an environment variable or provide it when prompted
# If you run this notebook locally, you may have to reload the terminal and the notebook to make the environment available

import os
from getpass import getpass

# Check if OPENAI_API_KEY is set as an environment variable
if os.getenv("OPENAI_API_KEY") is not None:
    print("Your OPENAI_API_KEY is ready")
else:
    # If not, prompt for it
    api_key = getpass("Enter your OPENAI_API_KEY: ")
    if api_key:
        print("Your OPENAI_API_KEY is now available for this session")
        # Optionally, you can set it as an environment variable for the current session
        os.environ["OPENAI_API_KEY"] = api_key
    else:
        print("You did not enter your OPENAI_API_KEY")

Your OPENAI_API_KEY is ready


## Neonデータベースに接続する

以下にNeonデータベースの接続文字列を入力するか、`DATABASE_URL`変数を使用して`.env`ファイルで定義してください。Neon接続文字列の取得方法については、[任意のアプリケーションから接続する](https://neon.tech/docs/connect/connect-from-any-app)を参照してください。

In [1]:
import os
import psycopg2
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# The connection string can be provided directly here.
# Replace the next line with Your Neon connection string.
connection_string = "postgres://<user>:<password>@<hostname>/<dbname>"

# If connection_string is not directly provided above, 
# then check if DATABASE_URL is set in the environment or .env.
if not connection_string:
    connection_string = os.environ.get("DATABASE_URL")

    # If neither method provides a connection string, raise an error.
    if not connection_string:
        raise ValueError("Please provide a valid connection string either in the code or in the .env file as DATABASE_URL.")

# Connect using the connection string
connection = psycopg2.connect(connection_string)

# Create a new cursor object
cursor = connection.cursor()

データベースへの接続をテストしてください：

In [2]:
# Execute this query to test the database connection
cursor.execute("SELECT 1;")
result = cursor.fetchone()

# Check the query result
if result == (1,):
    print("Your database connection was successful!")
else:
    print("Your connection failed.")

Your database connection was successful!


このガイドでは、OpenAI Cookbook の `examples` ディレクトリで利用可能な事前計算済みのWikipedia記事埋め込みを使用します。これにより、あなた自身のOpenAIクレジットを使って埋め込みを計算する必要がありません。

事前計算済みの埋め込みzipファイルをインポートしてください：

In [6]:
import wget

embeddings_url = "https://cdn.openai.com/API/examples/data/vector_database_wikipedia_articles_embedded.zip"

# The file is ~700 MB. Importing it will take several minutes.
wget.download(embeddings_url)

'vector_database_wikipedia_articles_embedded.zip'

ダウンロードしたzipファイルを展開してください：

In [7]:
import zipfile
import os
import re
import tempfile

current_directory = os.getcwd()
zip_file_path = os.path.join(current_directory, "vector_database_wikipedia_articles_embedded.zip")
output_directory = os.path.join(current_directory, "../../data")

with zipfile.ZipFile(zip_file_path, "r") as zip_ref:
    zip_ref.extractall(output_directory)


# Check to see if the csv file was extracted
file_name = "vector_database_wikipedia_articles_embedded.csv"
data_directory = os.path.join(current_directory, "../../data")
file_path = os.path.join(data_directory, file_name)


if os.path.exists(file_path):
    print(f"The csv file {file_name} exists in the data directory.")
else:
    print(f"The csv file {file_name} does not exist in the data directory.")

The file vector_database_wikipedia_articles_embedded.csv exists in the data directory.


## ベクトル埋め込み用のテーブルを作成し、インデックスを追加する

データベースに作成されるベクトルテーブルは**articles**と呼ばれます。各オブジェクトには**title**と**content**のベクトルがあります。

**title**と**content**の両方のベクトル列にインデックスが定義されています。

In [3]:
create_table_sql = '''
CREATE TABLE IF NOT EXISTS public.articles (
    id INTEGER NOT NULL,
    url TEXT,
    title TEXT,
    content TEXT,
    title_vector vector(1536),
    content_vector vector(1536),
    vector_id INTEGER
);

ALTER TABLE public.articles ADD PRIMARY KEY (id);
'''

# SQL statement for creating indexes
create_indexes_sql = '''
CREATE INDEX ON public.articles USING ivfflat (content_vector) WITH (lists = 1000);

CREATE INDEX ON public.articles USING ivfflat (title_vector) WITH (lists = 1000);
'''

# Execute the SQL statements
cursor.execute(create_table_sql)
cursor.execute(create_indexes_sql)

# Commit the changes
connection.commit()

## データの読み込み

事前に計算されたベクトルデータを`.csv`ファイルから`articles`テーブルに読み込みます。25,000件のレコードがあるため、この操作には数分かかることが予想されます。

In [4]:
import io

# Path to your local CSV file
csv_file_path = '../../data/vector_database_wikipedia_articles_embedded.csv'

# Define a generator function to process the csv file
def process_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            yield line

# Create a StringIO object to store the modified lines
modified_lines = io.StringIO(''.join(list(process_file(csv_file_path))))

# Create the COPY command for copy_expert
copy_command = '''
COPY public.articles (id, url, title, content, title_vector, content_vector, vector_id)
FROM STDIN WITH (FORMAT CSV, HEADER true, DELIMITER ',');
'''

# Execute the COPY command using copy_expert
cursor.copy_expert(copy_command, modified_lines)

# Commit the changes
connection.commit()

データが読み込まれていることを確認するため、レコード数をチェックしてください。25000件のレコードがあるはずです。

In [5]:
# Check the size of the data
count_sql = """select count(*) from public.articles;"""
cursor.execute(count_sql)
result = cursor.fetchone()
print(f"Count:{result[0]}")

Count:25000


## データを検索する

データがNeonデータベースに保存された後、最近傍のデータをクエリできます。

まず、`query_neon`関数を定義します。この関数は、ベクトル類似度検索を実行する際に実行されます。この関数は、ユーザーのクエリに基づいて埋め込みを作成し、SQLクエリを準備し、その埋め込みを使用してSQLクエリを実行します。データベースに読み込んだ事前計算済みの埋め込みは、`text-embedding-3-small` OpenAIモデルで作成されたため、類似度検索用の埋め込みを作成する際も同じモデルを使用する必要があります。

「title」または「content」に基づいて検索できる`vector_name`パラメータが提供されています。

In [6]:
def query_neon(query, collection_name, vector_name="title_vector", top_k=20):

    # Create an embedding vector from the user query
    embedded_query = openai.Embedding.create(
        input=query,
        model="text-embedding-3-small",
    )["data"][0]["embedding"]

    # Convert the embedded_query to PostgreSQL compatible format
    embedded_query_pg = "[" + ",".join(map(str, embedded_query)) + "]"

    # Create the SQL query
    query_sql = f"""
    SELECT id, url, title, l2_distance({vector_name},'{embedded_query_pg}'::VECTOR(1536)) AS similarity
    FROM {collection_name}
    ORDER BY {vector_name} <-> '{embedded_query_pg}'::VECTOR(1536)
    LIMIT {top_k};
    """
    # Execute the query
    cursor.execute(query_sql)
    results = cursor.fetchall()

    return results

`title_vector`埋め込みに基づいて類似性検索を実行します：

In [7]:
# Query based on `title_vector` embeddings
import openai

query_results = query_neon("Greek mythology", "Articles")
for i, result in enumerate(query_results):
    print(f"{i + 1}. {result[2]} (Score: {round(1 - result[3], 3)})")

1. Greek mythology (Score: 0.998)
2. Roman mythology (Score: 0.7)
3. Greek underworld (Score: 0.637)
4. Mythology (Score: 0.635)
5. Classical mythology (Score: 0.629)
6. Japanese mythology (Score: 0.615)
7. Norse mythology (Score: 0.569)
8. Greek language (Score: 0.566)
9. Zeus (Score: 0.534)
10. List of mythologies (Score: 0.531)
11. Jupiter (mythology) (Score: 0.53)
12. Greek (Score: 0.53)
13. Gaia (mythology) (Score: 0.526)
14. Titan (mythology) (Score: 0.522)
15. Mercury (mythology) (Score: 0.521)
16. Ancient Greece (Score: 0.52)
17. Greek alphabet (Score: 0.52)
18. Venus (mythology) (Score: 0.515)
19. Pluto (mythology) (Score: 0.515)
20. Athena (Score: 0.514)


`content_vector`埋め込みに基づいて類似性検索を実行する：

In [8]:
# Query based on `content_vector` embeddings
query_results = query_neon("Famous battles in Greek history", "Articles", "content_vector")
for i, result in enumerate(query_results):
    print(f"{i + 1}. {result[2]} (Score: {round(1 - result[3], 3)})")

1. 222 BC (Score: 0.489)
2. Trojan War (Score: 0.458)
3. Peloponnesian War (Score: 0.456)
4. History of the Peloponnesian War (Score: 0.449)
5. 430 BC (Score: 0.441)
6. 168 BC (Score: 0.436)
7. Ancient Greece (Score: 0.429)
8. Classical Athens (Score: 0.428)
9. 499 BC (Score: 0.427)
10. Leonidas I (Score: 0.426)
11. Battle (Score: 0.421)
12. Greek War of Independence (Score: 0.421)
13. Menelaus (Score: 0.419)
14. Thebes, Greece (Score: 0.417)
15. Patroclus (Score: 0.417)
16. 427 BC (Score: 0.416)
17. 429 BC (Score: 0.413)
18. August 2 (Score: 0.412)
19. Ionia (Score: 0.411)
20. 323 (Score: 0.409)
